In [158]:
import os
import json
import pandas as pd
import traceback
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks import get_openai_callback
from langchain.chains import SequentialChain


In [159]:
from langchain.chat_models import ChatOpenAI

In [160]:
KEY = os.getenv("OPEN_AI_KEY")

In [161]:
llm = ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.5)

In [162]:
TEMPLATE = """
  Text:{text}
  You are an expert MCQ maker. Given the above text, it is your job to\
  create a quiz of {number} multiple choice questions(MCQ) of a {subject}. In a {tone} tone.\
  Make sure the questions are not repeated and check all the questions to be conforming the text as well.
  Make sure to format your responses like RESPONSE_JSON below and use it as a guide.\
  Ensure to make {number} of MCQs
  {response_json}
"""

In [163]:
RESPONSE_JSON = {
  "1":{
    "mcq":"multiple choice question",
    "options":{
      "A": " choice here",
      "B": " choice here",
      "C": " choice here",
      "D": " choice here"
    },
    "correct":" correct answer letter"
    },
  "2":{
    "mcq":"multiple choice question",
    "options":{
      "A": " choice here",
      "B": " choice here",
      "C": " choice here",
      "D": " choice here"
    },
    "correct":" correct answer letter"
    },
  "3":{
    "mcq":"multiple choice question letter",
    "options":{
      "A": " choice here",
      "B": " choice here",
      "C": " choice here",
      "D": " choice here"
  },
    "correct":" correct answer"
  }
}

In [164]:
quiz_generation_prompt = PromptTemplate(
  input_variables = ["text","number","subject","tone","response_json"],
  template=TEMPLATE
)

In [165]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [166]:
TEMPLATE2 = """
  you are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} for students\
  You need to evaluate the complexity of the questions and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis,
  if the quiz is not at per with the cognitive and analytical abilities of the students,\
  update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student.
  Quiz_MCQs:
  {quiz}
  
  Check from and expert English Writer of the above quiz:
"""

In [167]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject","quiz"],template=TEMPLATE2)

In [168]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [169]:
generate_evaluate_chain = SequentialChain(chains=[quiz_chain,review_chain],input_variables = ["text","number","subject","tone","response_json"],
                                          output_variables = ["quiz","review"], verbose=True)

In [170]:
file_path = r"C:\Users\user\Documents\Github\projects\openAi-langchain\mcqGenerator-project\data.txt"

In [171]:
file_path

'C:\\Users\\user\\Documents\\Github\\projects\\openAi-langchain\\mcqGenerator-project\\data.txt'

In [172]:
with open(file_path,"r") as file:
  TEXT = file.read()

In [173]:
print(TEXT)

Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised learning.[6][7]

From a theoretical 

In [174]:
NUMBER=5,
SUBJECT = "machine learning",
TONE ="simple"

In [175]:
with get_openai_callback() as cb:
  response=generate_evaluate_chain(
    {
      "text":TEXT,
      "number":NUMBER,
      "subject":SUBJECT,
      "tone":TONE,
      "response_json":json.dumps(RESPONSE_JSON)
    }
  )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

  Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field o

In [176]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")

Total Tokens: 2117
Prompt Tokens: 1656
Completion Tokens: 461
Total Cost: 0.003406


In [177]:
response

{'text': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field\'s methods.\n\nThe mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised learning.[6][7]\n\nFr

In [178]:
quiz = json.loads(response.get("quiz"))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [151]:
type(quiz)

dict

In [152]:
quiz_table_data = []
for key,value in quiz.items():
  mcq = value["mcq"]
  options = " | ".join (
    [
      f"{option}:{option_value}"
      for option, option_value in value["options"].items()
    ] )
  correct = value["correct"]
  quiz_table_data.append({"MCQ":mcq,"Choices":options, "Correct":correct})

In [153]:
quiz_table_data

[{'MCQ': 'What is machine learning?',
  'Choices': 'A:A field of study in artificial intelligence concerned with the development and study of statistical algorithms | B:A field of study in computer networking | C:A field of study in mechanical engineering | D:A field of study in environmental science',
  'Correct': 'A'},
 {'MCQ': 'Which mathematical foundations provide the basis for machine learning?',
  'Choices': 'A:Algebra | B:Calculus | C:Mathematical optimization methods | D:Geometry',
  'Correct': 'C'},
 {'MCQ': "Who coined the term 'machine learning'?",
  'Choices': 'A:Donald Hebb | B:Tom M. Mitchell | C:Arthur Samuel | D:Walter Pitts',
  'Correct': 'C'},
 {'MCQ': 'What is the purpose of machine learning algorithms in stock trading?',
  'Choices': 'A:To predict future outcomes based on models | B:To classify data based on models | C:To analyze natural language processing | D:To study human cognitive processes',
  'Correct': 'A'},
 {'MCQ': 'What is the framework for describing ma

In [156]:
quiz = pd.DataFrame(quiz_table_data)

In [157]:
quiz.to_csv("machinelearning.csv",index=False)